<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/NeuralNets/CNN_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [181]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import sys
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from PIL import Image
import PIL
print(Image.__file__)
import cv2
import glob
import pandas as pd
import imageio

/usr/local/lib/python3.7/dist-packages/PIL/Image.py


## Load Data + Preprocessing

In [ ]:
def extract_images(path, c): # path of data, class of data
  filename_arr = []
  X_arr = []

  fixed_height = 100

  for file in glob.glob(path):
    filename_arr.append(file) # filenames, not going to use them for now. Might need them later.
    image = Image.open(file)
    height_percent = (fixed_height / float(image.size[1]))
    width_size = int((float(image.size[0]) * float(height_percent)))
    image = image.resize((width_size, fixed_height), PIL.Image.NEAREST)
    image.save(file)

    x = cv2.imread(file)
    # print(x.shape)
    x = x[:,:,0] # removing all color channels to make data smaller and possible to use
    X_arr.append(x)

  y_arr = [c] * len(X_arr)

  return X_arr, y_arr

In [ ]:
def pad_images(array):
  arr = array.copy()
  largestX = 0
  largestY = 0

  def pad_condition(pad, largest, index):
    if (2 * pad != (largest - arr[i].shape[index])):
      pad1 = pad + 1
      pad2 = pad
    else:
      pad1, pad2 = pad, pad
    return pad1, pad2

  for i in arr:
    X = i.shape[0]
    Y = i.shape[1]
    if (X > largestX):
      largestX = X
    if (Y > largestY):
      largestY = Y

  for i in range(len(arr)):
    X_pad = int((largestX - arr[i].shape[0]) /2) # pad equally in both directions, must be int
    Y_pad = int((largestY - arr[i].shape[1]) /2)
    
    # but int floors, so we might get something of a slightly wrong shape (by 1), so...
    X_pad1, X_pad2 = pad_condition(X_pad, largestX, 0)
    Y_pad1, Y_pad2 = pad_condition(Y_pad, largestY, 1)

    arr[i] = np.pad(arr[i], ((X_pad1, X_pad2), (Y_pad1, Y_pad2)), 'constant', constant_values=(0)) # each image in arr in 2D
    
  maxSize = largestX * largestY
  
  return arr, maxSize

In [ ]:
def normalize_images(arr, num):
  arr = np.array(arr)/num
  return arr

In [ ]:
# Old Hand PD image extraction
X_meah, y_meah = extract_images("/content/drive/MyDrive/Data/Images/Old_Meander/MeanderControl/*.*",0)
X_meap, y_meap = extract_images("/content/drive/MyDrive/Data/Images/Old_Meander/MeanderPatients/*.*",1)

X_meander = X_meah + X_meap
y_meander = y_meah + y_meap

X_spih, y_spih = extract_images("/content/drive/MyDrive/Data/Images/Old_Spiral/SpiralControl/*.*",0)
X_spip, y_spip = extract_images("/content/drive/MyDrive/Data/Images/Old_Spiral/SpiralPatients/*.*",1)

X_spiral = X_spih + X_spip
y_spiral = y_spih + y_spip

In [ ]:
# New Hand PD image extraction, ignoring all PD patient data (what we did to fix class imbalance in SVM)

X_new_meah, y_new_meah = extract_images("/content/drive/MyDrive/Data/Images/New_Meander/HealthyMeander/HealthyMeander/*.*",0)
# X_new_meap, y_new_meap = extract_images("/content/drive/MyDrive/Data/Images/New_Meander/PatientMeander/PatientMeander/*.*",1)

X_new_meander = X_new_meah # + X_new_meap
y_new_meander = y_new_meah # + y_new_meap

X_new_spih, y_new_spih = extract_images("/content/drive/MyDrive/Data/Images/New_Spiral/HealthySpiral/HealthySpiral/*.*",0)
# X_new_spip, y_new_spip = extract_images("/content/drive/MyDrive/Data/Images/New_Spiral/PatientSpiral/PatientSpiral/*.*",1)

X_new_spiral = X_new_spih # + X_new_spip
y_new_spiral = y_new_spih # + y_new_spip

In [ ]:
X_all = X_spiral + X_new_spiral + X_meander + X_new_meander
X_all, maxSize = pad_images(X_all)
X_all = normalize_images(X_all, 255)

In [ ]:
y_all = np.array(y_spiral + y_new_spiral + y_meander + y_new_meander)

In [ ]:
y_all_series = pd.Series(y_all)
y_all_series.value_counts()

1    592
0    424
dtype: int64

In [ ]:
len(y_all)

1016

In [ ]:
maxSize

13700

### Train-Test-Split

Consider random state and possibility for different train test validation sets.

In [ ]:
y_label = y_all.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state = 0, test_size = 0.15, stratify = y_label)

In [ ]:
len(y_train), len(y_test)

(863, 153)

In [ ]:
# consider setting test_size to larger 

train_labels = y_train.copy()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 0, test_size = 0.1, stratify = train_labels)

In [ ]:
len(y_train), len(y_val)

(776, 87)

In [ ]:
X_train[0].shape

(100, 137)

In [ ]:
y_train[0]

0

In [182]:
# sequence level storage, no patient level rn

def save_to_directories(X, y, path):
  
  # convert numpy array to image
  for i in range(len(X)):
    label = y[i]

    # find image's label, control or patient
    if label == 1:
      # save image to correct directory
      path_pd = path + "/Patient/image_{}.jpg".format(i+1)
      imageio.imwrite(path_pd, X[i])

    elif label == 0:
      path_control = path + "/Control/image_{}.jpg".format(i+1)
      imageio.imwrite(path_control, X[i])

In [185]:
save_to_directories(X_train, y_train, "/content/drive/MyDrive/Data/Images/Split_Data/Train")

In [ ]:
WIDTH = 137
HEIGHT = 100

In [ ]:
train_data = ImageDataGenerator()
val_data = ImageDataGenerator()
test_data = ImageDataGenerator()

train_generator = train_data.flow_from_directory(
    train_dir, 
    target_size = (WIDTH, HEIGHT), 
    batch_size = 4, 
    class_mode = 'binary' 
)

In [ ]:
val_generator = val_data.flow_from_directory(
    val_dir, 
    target_size = (WIDTH, HEIGHT), 
    batch_size = 4, 
    class_mode = 'binary' 
)

test_generator = test_data.flow_from_directory(
    test_dir, 
    target_size = (WIDTH, HEIGHT), 
    batch_size = 4, 
    class_mode = 'binary' 
)

In [ ]:
model = tf.keras.models.Sequential([
    #conv net part
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)), #3 is for rgb color channels
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    #dnn part
    tf.keras.layers.Flatten(),
   
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),

    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
class callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
    #creating a callback function that activates if the accuracy is greater than 60%
        if (logs.get('accuracy') > 0.96):
          print("Stopping Training")
          self.model.stop_training = True
            
callbacks = callback()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr = 0.0001),
              metrics=['accuracy'])

In [ ]:
#training model after loading data in using an image generator
history = model.fit (
    #training part
    train_generator,
    steps_per_epoch = 4, 
    epochs = 15,
    
    #testing part
    validation_data = val_generator,
    validation_steps = 4,
    callbacks =[callbacks],
    verbose = 1
)